In [ ]:
from tqdm.notebook import tqdm
import logging
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(format=' %(asctime)s - %(levelname)s - %(message)s')

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [ ]:
from contextlib import ExitStack
from waiting import wait

from tapearchive import app
from tapearchive.utils import get_config

app_config = get_config("./config.yaml")
exit_stack = ExitStack()
connection_pool = app.create_db_connection(app_config)
dispatcher = app.create_dispatcher(connection_pool, app_config, exit_stack)

In [ ]:
from tapearchive.workflow.tasks.audio_analisis import FindTuneKey
from tapearchive.models.raw_data import FileDao

import pathlib 
import glob

source_path = pathlib.Path("/home/caiwan/hdd/**/*.*")\

file_dao = FileDao(connection_pool)

for file in tqdm(glob.glob(str(source_path), recursive=True)):
    input_file = pathlib.Path(file)
    if input_file.is_file():
        file_id = file_dao.pull_from_disk(input_file)

        print(input_file)
        
        dispatcher.post_task(FindTuneKey(
            source_file_id=file_id,
            source_format=input_file.suffix
        ))


        break

In [ ]:
logger.debug("------------")

dispatcher.terminate()
wait(lambda: dispatcher.is_exit)

exit_stack.close()